In [1]:
#!#/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 26 17:27:38 2020

@author: syook
"""
############## import modules ##############
from __future__ import print_function

import os

from keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
# from livelossplot.tf_keras import PlotLossesCallback
# from livelossplot import PlotLossesKeras

import pandas as pd # for using pandas daraframe
import numpy as np # for som math operations

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import cys_loading
import cys_utils
#############################################


############# GPU selection #################
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#############################################

# load train_data, train_label
binary_h_o = cys_loading.binary_h_o()
train_hypnos = binary_h_o.load_input()
train_labels = binary_h_o.load_label() # 1: healthy, 0: OSA

# train_test_split
print("train_test_splie --> split rate=0.1")
x_train, x_test, y_train, y_test = train_test_split(train_hypnos, train_labels, test_size=0.1, random_state=42)


#############################################
########### kernel size experiment ##########
list_kernel_size = [(2,2)]
for j in range(2,10):
    ker_nel_len = j*5
    list_kernel_size.append((2,ker_nel_len))
#############################################

test_acc=[]

Using TensorFlow backend.
c:\Users\CYS\Anaconda3\envs\yook\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\CYS\Anaconda3\envs\yook\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\CYS\Anaconda3\envs\yook\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\CYS\Anaconda3\envs\yook\lib\site-packages\tenso

train_test_splie --> split rate=0.1


In [2]:
# from livelossplot import PlotLossesKerasTF
# from livelossplot.outputs import MatplotlibPlot
############# Train model ###################
# gpu memory 부족으로 인해서, for문을 이용해서 한 번에 실행하는 것은 불가능함
# i를 하나씩 바꿔주면서 실행하자
i = 2
model = cys_utils.get_model(kernel_size=list_kernel_size[i])

path_weights = './weights/osa_%s_weights' % str(list_kernel_size[i])
# checkpoint = ModelCheckpoint(path_weights, monitor='val_loss', save_best_only=True,
#                             save_weights_only=True, mode='min', verbose=0)
checkpoint = ModelCheckpoint(path_weights, mode='min', verbose=0)

# callbacks = [PlotLossesCallback(), checkpoint]#, reduce_lr]
# save the train_loss plot
path_figure = './figure/osa_%s.png' % str(list_kernel_size[i])
# callbacks = [PlotLossesKerasTF(outputs=[MatplotlibPlot(figpath=path_figure)]), checkpoint]
callbacks = [checkpoint]

batch_size=32
epochs=10

H = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    callbacks=callbacks,
    validation_split=0.1,
    epochs=epochs
    # class_weight=class_weight
    )

cys_utils.plot_acc_loss(H, path_figure, epochs)

y_true = y_test
y_pred = np.argmax(model.predict(x_test), 1)

print(metrics.classification_report(y_true, y_pred))
# df_metrics = pd.DataFrame(metrics.classification_report(y_true, y_pred))
# df_metrics.to_csv('./metric/osa_%s.csv' % str(list_kernel_size))

plt.figure()
labels = ['OSA', 'Healthy']
ax= plt.subplot()
sns.heatmap(metrics.confusion_matrix(y_true, y_pred, normalize='true'), annot=True, ax = ax, cmap=plt.cm.Blues); #annot=True to annotate cells
# labels, title and ticks
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); ax.yaxis.set_ticklabels(labels);
plt.savefig('./confusion_matrix/osa_%s.png' % str(list_kernel_size[i]))

test_acc.append(metrics.accuracy_score(y_true, y_pred))
  

now kernel size: (2, 15)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 5, 660, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 660, 64)   1984        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 660, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 5, 660, 64)   0           batch_normalization_1[0][0]      
_________________________________________________________________

OSError: [Errno 22] Unable to create file (unable to open file: name = './weights/osa_(2, 15)_wights', errno = 22, error message = 'Invalid argument', flags = 13, o_flags = 302)

In [ ]:
test_acc = []
test_acc.append(metrics.accuracy_score(y_true, y_pred))
print("Classification accuracy: %0.6f" % metrics.accuracy_score(y_true, y_pred))
# first iteration acc: 0.607930
# second: 0.6431718
# 3: 0.788546
# 4: 

In [ ]:
test_acc

In [ ]:
# # from livelossplot import PlotLossesKerasTF
# # from livelossplot.outputs import MatplotlibPlot
# ############# Train model ###################
# for i in tqdm(range(len(list_kernel_size))):
#     model = cys_utils.get_model(kernel_size=list_kernel_size[i])

#     path_weights = './weights/osa_%s_wights' % str(list_kernel_size[i])
#     # checkpoint = ModelCheckpoint(path_weights, monitor='val_loss', save_best_only=True,
#     #                             save_weights_only=True, mode='min', verbose=0)
#     checkpoint = ModelCheckpoint(path_weights, mode='min', verbose=0)
    
#     # callbacks = [PlotLossesCallback(), checkpoint]#, reduce_lr]
#     # save the train_loss plot
#     path_figure = './figure/osa_%s.png' % str(list_kernel_size[i])
#     # callbacks = [PlotLossesKerasTF(outputs=[MatplotlibPlot(figpath=path_figure)]), checkpoint]
#     callbacks = [checkpoint]
    
#     batch_size=32
#     epochs=3

#     H = model.fit(
#         x_train,
#         y_train,
#         batch_size=batch_size,
#         callbacks=callbacks,
#         validation_split=0.1,
#         epochs=epochs
#         # class_weight=class_weight
#         )
    
#     cys_utils.plot_acc_loss(H, path_figure, epochs)
    
#     y_true = y_test
#     y_pred = np.argmax(model.predict(x_test), 1)
    
#     print(metrics.classification_report(y_true, y_pred))
#     # df_metrics = pd.DataFrame(metrics.classification_report(y_true, y_pred))
#     # df_metrics.to_csv('./metric/osa_%s.csv' % str(list_kernel_size))

#     plt.figure()
#     labels = ['OSA', 'Healthy']
#     ax= plt.subplot()
#     sns.heatmap(metrics.confusion_matrix(y_true, y_pred, normalize='true'), annot=True, ax = ax, cmap=plt.cm.Blues); #annot=True to annotate cells
#     # labels, title and ticks
#     ax.set_title('Confusion Matrix'); 
#     ax.xaxis.set_ticklabels(labels); ax.yaxis.set_ticklabels(labels);
#     plt.savefig('./confusion_matrix/osa_%s.png' % str(list_kernel_size[i]))
  